In [1]:
import os

# Step 1: Import Modules
from DataLoader import ADE20KDownloader
from DataHandler import DataHandler 
from DeepLabV3Plus import DeepLabV3Plus  # The DeepLabV3+ model implementation
from Trainer import Trainer  # The Trainer class to handle the training process
from DisplayCallback import DisplayCallback  # The display callback for visualization
from ModelEvaluator import ModelEvaluator

IMG_SIZE = 256 # Replace with your image size
N_CLASSES = 21  # Replace with the number of classes in your dataset

BATCH_SIZE = 1

BASE_DIR = os.getcwd()
DATASET_PATH = 'ADEChallengeData2016/images'
DOWNLOAD_PATH = 'path_to_download'
DOWNLOAD_PATH = os.path.join(BASE_DIR, DOWNLOAD_PATH)
DATASET_PATH = os.path.join(DOWNLOAD_PATH, DATASET_PATH)
downloader = ADE20KDownloader(DOWNLOAD_PATH)
downloader.download_ade(overwrite=False)  # Overwrite existing files if set to True

/Users/sean/v-newtf/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
import tensorflow as tf
import numpy as np
from glob import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array

 
training_data = "training/"
val_data = "validation/"
SEED = 42
AUTOTUNE = tf.data.experimental.AUTOTUNE
TRAINSET_SIZE = len(glob( os.path.join(DATASET_PATH,training_data) + "*.jpg"))
print(f"The Training Dataset contains {TRAINSET_SIZE} images.")

VALSET_SIZE = len(glob(os.path.join(DATASET_PATH,val_data)+ "*.jpg"))
print(f"The Validation Dataset contains {VALSET_SIZE} images.")


def parse_image(img_path: str) -> dict:
    """Load an image and its annotation (mask) and returning
    a dictionary.

    Parameters
    ----------
    img_path : str
        Image (not the mask) location.

    Returns
    -------
    dict
        Dictionary mapping an image and its annotation.
    """
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.uint8)

    # For one Image path:
    # .../trainset/images/training/ADE_train_00000001.jpg
    # Its corresponding annotation path is:
    # .../trainset/annotations/training/ADE_train_00000001.png
    mask_path = tf.strings.regex_replace(img_path, "images", "annotations")
    mask_path = tf.strings.regex_replace(mask_path, "jpg", "png")
    mask = tf.io.read_file(mask_path)
    # The masks contain a class index for each pixels
    mask = tf.image.decode_png(mask, channels=1)
    # In scene parsing, "not labeled" = 255
    # But it will mess up with our N_CLASS = 150
    # Since 255 means the 255th class
    # Which doesn't exist
    mask = tf.where(mask == 255, np.dtype('uint8').type(0), mask)
    # Note that we have to convert the new value (0)
    # With the same dtype than the tensor itself

    return {'image': image, 'segmentation_mask': mask}


train_dataset = tf.data.Dataset.list_files( os.path.join(DATASET_PATH, training_data) + "*.jpg", seed=SEED)
train_dataset = train_dataset.map(parse_image)

val_dataset = tf.data.Dataset.list_files(os.path.join(DATASET_PATH, val_data) + "*.jpg", seed=SEED)
val_dataset =val_dataset.map(parse_image)

import numpy as np
import csv

def create_ade20k_label_colormap(file_path):
    """Creates a label colormap used in ADE20K segmentation benchmark from a file.

    Args:
      file_path: The path to the CSV file containing the colormap.

    Returns:
      A colormap for visualizing segmentation results.
    """
    colormap = []
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            colormap.append([int(c) for c in row])

    return np.asarray(colormap)

# Example usage:
# Make sure to have a file 'ade20k_colormap.csv' with the color values.
file_path = 'ade20k_colormap.csv'
ade20k_colormap = create_ade20k_label_colormap(file_path)

with open("objectInfo151.txt", "r") as file:
    lines = file.readlines()[1:152]

class_names = []
pixel_count_dict = {}
color_dict = {}  # This dictionary will store the color for each class
for idx, line in enumerate(lines):
    parts = line.split('\t')
    class_idx = int(parts[0])
    class_name = parts[4].strip()
    class_names.append(class_name)

     
    color_dict[class_name] = ade20k_colormap[idx]
  
N_TOTAL_CLASSES = len(class_names)
 
CLASSES_COLOR_DICT = color_dict  
#name_to_index_dict = {name: index for index, name in enumerate(class_names)}
name_to_index_dict = {name: index for index, name in enumerate(class_names, start=0)}
 
 
@tf.function
def normalize(input_image: tf.Tensor, input_mask: tf.Tensor) -> tuple:
     
    input_image = tf.cast(input_image, tf.float32) / 255.0
    return input_image, input_mask

@tf.function
def load_image_train(datapoint: dict) -> tuple:
     
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask) 
        
    input_image = tf.image.random_brightness(input_image, max_delta=0.3)  
    input_image = tf.image.random_contrast(input_image, lower=0.8, upper=1.2)   

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

@tf.function
def load_image_test(datapoint: dict) -> tuple:
   
    input_image = tf.image.resize(datapoint['image'], (IMG_SIZE, IMG_SIZE))
    input_mask = tf.image.resize(datapoint['segmentation_mask'], (IMG_SIZE, IMG_SIZE))

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

def filter_label(image, label):
    label = tf.where(tf.equal(label, N_TOTAL_CLASSES), tf.cast(tf.constant(0), label.dtype), label)  
    return image, label

 
BUFFER_SIZE = 1000

dataset = {"train": train_dataset, "val": val_dataset}

# -- Train Dataset --#
dataset['train'] = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset['train'] = dataset['train'].map(filter_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)   
dataset['train'] = dataset['train'].shuffle(buffer_size=BUFFER_SIZE, seed=SEED)
#dataset['train'] = dataset['train'].repeat()
dataset['train'] = dataset['train'].batch(BATCH_SIZE)
dataset['train'] = dataset['train'].prefetch(buffer_size=AUTOTUNE)

#-- Validation Dataset --#
dataset['val'] = dataset['val'].map(load_image_test)
dataset['val'] = dataset['val'].map(filter_label)  # 추가
#dataset['val'] = dataset['val'].repeat()
dataset['val'] = dataset['val'].batch(BATCH_SIZE)
dataset['val'] = dataset['val'].prefetch(buffer_size=AUTOTUNE)


The Training Dataset contains 20210 images.
The Validation Dataset contains 2000 images.


2023-11-05 17:51:28.475005: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2023-11-05 17:51:28.475030: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2023-11-05 17:51:28.475035: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2023-11-05 17:51:28.475070: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-05 17:51:28.475084: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Your code to create and train a model...
# model = create_model()
# train_model(model, train_dataset, val_dataset)

# Step 3: Initialize the Model
# Set the image size and the number of classes based on your dataset

deeplab = DeepLabV3Plus(image_size=IMG_SIZE, num_classes=N_CLASSES)


In [4]:
# Assume train_images, train_masks, val_images, val_masks are already prepared.
  
trainer = Trainer(model=deeplab.model, dataset=dataset, batch_size=BATCH_SIZE)

# Step 5: Start Training
model_history = trainer.train()

# Optionally, visualize the results after training
# Visualization would depend on what your 'show_predictions()' function does
# For example, if you have a test image and mask, you could do:
# test_img, test_mask = data_loader.get_test_sample()  # Assuming this method exists
# predicted_mask = model.predict(test_img)
# display_segmentation(test_img, test_mask, predicted_mask)  # Assuming this function exists

{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>, 'val': <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>}
Epoch 1/500


/Users/sean/v-newtf/lib/python3.9/site-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-11-05 17:51:32.477923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


    3/20210 [..............................] - ETA: 1:15:38 - loss: 2.7946 - accuracy: 0.0388

KeyboardInterrupt: 

In [5]:
model_evaluator = ModelEvaluator(deeplab.model, dataset, name_to_index_dict, CLASSES_COLOR_DICT)
model_evaluator.evaluate()

2023-11-05 17:51:38.379865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


sky_count: 736
Average Sky IoU: 0.0000
Average mIoU: 0.0000
